## Retrieval of Article IDs from Natural Language Query

Most of the code here has been abstracted away into pubmed_search, thus all are commented out.

In [1]:
# import requests
# import spacy
# from nltk.corpus import stopwords

# # Load SpaCy's language model
# nlp = spacy.load("en_core_web_sm")

# # Define stop words using NLTK
# stop_words = set(stopwords.words('english'))

# def remove_stop_words(query):
#     doc = nlp(query)
#     # Filter out stop words and only keep nouns and adjectives
#     filtered_tokens = [token.text for token in doc if token.pos_ in ["NOUN", "ADJ"] and token.text.lower() not in stop_words]
#     return " ".join(filtered_tokens)

# def get_query_response(query, preprocess = True, articles_to_retrieve = 5):
#     if preprocess:
#         query = remove_stop_words(query)
#         print('Preprocessed query:', query)
#     base_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
#     params = {
#     "db": "pubmed",        # Database to search
#     "term": query,  # Your natural language query
#     "retmax": articles_to_retrieve,         # Number of results to return
#     'api_key' : '2016449eab49266b3ccf1be9ba0c52b8d809',
#     "retmode": "json" ,     # Response format    
#     'maxdate':"2022/12/30"
#     }
#     response = requests.get(base_url, params=params)
#     if response.status_code == 200:
#         # Parse the JSON response
#         data = response.json()
#     else:
#         print(f"Error: {response.status_code}")
#     return data

# # response = requests.get(url= 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term=science[journal]+AND+breast+cancer+AND+2008[pdat]&retmode=json&api_key=2016449eab49266b3ccf1be9ba0c52b8d809')

The next methods extract article details from extracted ids.

In [2]:
# # Parsing the XML response
# from xml.etree import ElementTree as ET

# def get_article_details_from_id(id_list, important_headings_only = True):
#     efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
#     efetch_params = {
#         "db": "pubmed",
#         "id": ",".join(id_list),  # Join PMIDs with commas
#         "retmode": "xml",               # Request XML format for easier parsing
#         "rettype": "abstract"           # Request abstract type
#     }

#     # Send the EFetch request
#     efetch_response = requests.get(efetch_url, params=efetch_params)

#     if efetch_response.status_code == 200:
#         root = ET.fromstring(efetch_response.content)
#         articles = root.findall(".//PubmedArticle")

#         # # Original extraction
#         # retrieved_abstracts = {}
#         # for article in articles:
#         #     pmid = article.findtext(".//PMID")
#         #     title = article.findtext(".//ArticleTitle")
#         #     abstract = article.findtext(".//AbstractText")
#         #     retrieved_abstracts[pmid] = {
#         #         'title': title,
#         #         'abstract': abstract
#         #     }
#         #     print(f"PMID: {pmid}\nTitle: {title}\nAbstract: {abstract}\n{'-'*80}\n")        
#         # return retrieved_abstracts

#         # Improved extraction
#         retrieved_data = {}
#         for article in articles:
#             pmid = article.findtext(".//PMID")
#             title = article.findtext(".//ArticleTitle")
#             abstract = article.findtext(".//AbstractText")            
#             # Extract all MeSH Headings and details for each article
#             mesh_headings = []
#             for mesh_heading in article.findall(".//MeshHeading/DescriptorName"):
#                 descriptor_name = mesh_heading.text
#                 descriptor_ui = mesh_heading.attrib['UI']     
#                 descriptor_major_topic = mesh_heading.attrib.get('MajorTopicYN', 'Not Found')
#                 # This one will extract only major article descriptors if important_headings_only is set to True
#                 if (important_headings_only and (descriptor_major_topic=='Y')) or not important_headings_only:
#                     mesh_heading_details = {
#                         'Descriptor': descriptor_name,
#                         'Descriptor UI': descriptor_ui,
#                         'Is Major Topic': descriptor_major_topic
#                     }
#                     mesh_headings.append(mesh_heading_details)
            
#             retrieved_data[pmid] = {
#                 'Title': title,
#                 'Abstract': abstract,
#                 'Mesh Headings': mesh_headings
#             }
#             # print(f"PMID: {pmid}\nTitle: {title}\nAbstract: {abstract}\nMeSH Headings: {', '.join(mesh_headings)}\n{'-'*80}\n")
#         return retrieved_data
#     else:
#         print(f"Error fetching articles: {efetch_response.status_code}")
#         return None

# # Run the function on extracted IDs (ideally, store in in vector DB and rank the retrieval)
# retrieved_article_details = get_article_details_from_id(retrieved_ids)
# retrieved_article_details

This section uses functions abstracted from the above section.

In [3]:
# import json
# import helper.pubmed_search as pubs

# with open('helper/descriptors.json') as json_file:
#     descriptors = json.load(json_file)
# with open('helper/concepts.json') as json_file:
#     concepts = json.load(json_file)
# with open('helper/terms.json') as json_file:
#     terms = json.load(json_file)

# query = 'I have purple rashes and cough'
# # query_response = pubs.get_query_response(query)
# retrieved_article_details, translated_query = pubs.extract_article_details_from_query(query, articles_to_retrieve=20, remove_stop_words=False)
# retrieved_article_details

## Semantic Query Suggestion (With Haystack)

Last time we used Haystack but now no longer - just straight away obtain embeddings to compare via Transformers.

In [4]:
# from helper.LLM_retrieval import RetrievalManager
# from helper.config import api_key
# from haystack import Document
# import pandas as pd

# openai_retrieval_manager = RetrievalManager(api_key)
# retrieval_manager = RetrievalManager(api_key, embedding_model='open source')

First level: Term suggestion directly from semantic similarity with extracted articles' MeSH headings.

In [5]:
# heading_entries = []
# for key, value in retrieved_article_details.items():
#     # Extract and embed all extracted headings
#     for mesh_heading in value['Mesh Headings']:
#         heading_entries.append(Document(content=descriptors[mesh_heading['Descriptor']]['Combined Description'], meta={'file_path': mesh_heading['Descriptor'], 'file_type': 'Headings'}))
#     title = value['Title'] or ''
#     abstract = value ['Abstract'] or ''
#     title_abstract = title + abstract
#     heading_entries.append(Document(content=title_abstract, meta={'file_path': key, 'file_type': 'Title and Abstract'}))

# retrieval_manager.read_individual_documents(heading_entries)
# openai_retrieval_manager.read_individual_documents(heading_entries)

Function for document purging.

In [6]:
# def purge_document_store(document_store):
#     list_of_ids = []
#     for document in document_store.filter_documents({"field": "meta.file_type", "operator": "==", "value": "Headings"}):
#         list_of_ids.append(document.to_dict()['id'])
#     document_store.delete_documents(list_of_ids)
#     # print('Post deletion document store size:', document_store.count_documents())

# print('Pre deletion document store size:', headings_retrieval_manager.document_store.count_documents())
# purge_document_store(headings_retrieval_manager.document_store)
# print('Post deletion document store size:', headings_retrieval_manager.document_store.count_documents())
# # headings_retrieval_manager.document_store.filter_documents({"field": "meta.file_type", "operator": "==", "value": "Headings"})

Filter for non openai model.

In [7]:
# answer = retrieval_manager.answer_user(query, filter={"field": "meta.file_type", "operator": "==", "value": "Headings"})['retriever_selected_candidates']
# # answer = retrieval_manager.answer_user(query)['retriever_selected_candidates']
# description_selected_descriptors = []
# for key, value in answer.items():
#     if value >= heading_threshold:
#         temp_dict = {
#             'Name': key,
#             'Description': descriptors[key]['Combined Description'],
#             'Suitability': value*100
#         }
#         description_selected_descriptors.append(temp_dict)
# headings_for_query_suggestion = pd.DataFrame(description_selected_descriptors)
# headings_for_query_suggestion

For openai model.

In [8]:
# heading_threshold = 0.1
# answer = openai_retrieval_manager.answer_user(query, filter={"field": "meta.file_type", "operator": "==", "value": "Headings"})['retriever_selected_candidates']
# # answer = retrieval_manager.answer_user(query)['retriever_selected_candidates']
# description_selected_descriptors = []
# for key, value in answer.items():
#     if value >= heading_threshold:
#         temp_dict = {
#             'Name': key,
#             'Description': descriptors[key]['Combined Description'],
#             'Suitability': value*100
#         }
#         description_selected_descriptors.append(temp_dict)
# headings_for_query_suggestion = pd.DataFrame(description_selected_descriptors)
# headings_for_query_suggestion

Term suggestion from semantic similarity with MeSH headings from most relevant articles.

In [9]:
# answer = retrieval_manager.answer_user(query, filter={"field": "meta.file_type", "operator": "==", "value": "Title and Abstract"})['retriever_selected_candidates']
# # answer = retrieval_manager.answer_user(query)['retriever_selected_candidates']
# shortlisted_articles = []
# for key, value in answer.items():
#     if value >= title_abstract_threshold:
#         temp_dict = {
#             'Name': key,
#             'Title': retrieved_article_details[key]['Title'],
#             # 'Abstract': retrieved_article_details[key]['Abstract'],
#             'Suitability': value*100
#         }
#         shortlisted_articles.append(temp_dict)
# articles_for_query_suggestion = pd.DataFrame(shortlisted_articles)
# articles_for_query_suggestion

For the openai model.

In [10]:
# answer = openai_retrieval_manager.answer_user(query, filter={"field": "meta.file_type", "operator": "==", "value": "Title and Abstract"})['retriever_selected_candidates']
# # answer = retrieval_manager.answer_user(query)['retriever_selected_candidates']
# title_abstract_threshold = 0.3
# shortlisted_articles = []
# for key, value in answer.items():
#     if value >= title_abstract_threshold:
#         temp_dict = {
#             'Name': key,
#             'Title': retrieved_article_details[key]['Title'],
#             # 'Abstract': retrieved_article_details[key]['Abstract'],
#             'Suitability': value*100
#         }
#         shortlisted_articles.append(temp_dict)
# articles_for_query_suggestion = pd.DataFrame(shortlisted_articles)
# articles_for_query_suggestion

And finally, extract out the terms we need for query suggestion and append it to the query.

In [11]:
# # headings_from_articles_for_query_suggestion = 
# for shortlisted_article in shortlisted_articles:
#     article_related_headings = retrieved_article_details[shortlisted_article['Name']]['Mesh Headings']

## Semantic Query Suggestion (With Transformers)

Define functions and import libraries.

In [12]:
# import pandas as pd
# import numpy as np
# from transformers import AutoTokenizer, AutoModel
# import torch
# from openai import OpenAI
# from helper.config import api_key
# from sklearn.metrics.pairwise import cosine_similarity


# def embed_mesh_headings(text, model='openai', api_key = ''):
#     '''Takes in a text and desired model as argument and outputs the embedding vector.'''
#     if model == 'openai':
#         client = OpenAI(api_key = api_key)
#         text = text.replace("\n", " ")
#         embeddings = np.array([client.embeddings.create(input = [text], model=model).data[0].embedding])
#     else:
#         model_name = model
#         tokenizer = AutoTokenizer.from_pretrained(model_name)
#         model = AutoModel.from_pretrained(model_name)
#         # Tokenize the text and get embeddings
#         inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             embeddings = model(**inputs).last_hidden_state.mean(dim=1).numpy()  # Use mean pooling for sentence embedding
#     return embeddings


# def embed_mesh_headings_preloaded_model(text, **kwargs):
#     '''Takes in a text and desired model as argument and outputs the embedding vector.
#     For each model, the corresponding details would also need to be passed in where necessary.'''
#     if 'client' not in kwargs: #if not an openai model
#         tokenizer = kwargs['tokenizer']
#         model = kwargs['model']
#         # Tokenize the text and get embeddings
#         inputs = tokenizer([text], return_tensors="pt", padding=True, truncation=True)
#         with torch.no_grad():
#             embeddings = model(**inputs).last_hidden_state.mean(dim=1).numpy()  # Use mean pooling for sentence embedding    
#     else:
#         text = text.replace("\n", " ")
#         embeddings = np.array([kwargs['client'].embeddings.create(input = [text], model="text-embedding-3-large").data[0].embedding])
#     return embeddings

Initialize models and test whether the models work.

In [13]:
# client = OpenAI(api_key = api_key)
# # model_name = "w601sxs/b1ade-embed"
# # model_name = "dmis-lab/biobert-v1.1"  
# model_name = "sentence-transformers/all-mpnet-base-v2"
# tokenizer = AutoTokenizer.from_pretrained(model_name)
# model = AutoModel.from_pretrained(model_name)

# text1 = "This is the first sentence."
# text2 = "This is another sentence."
# # # Old method that initializes the model inside the function
# # cosine_sim = cosine_similarity(embed_mesh_headings(text1, "w601sxs/b1ade-embed" ), embed_mesh_headings(text2, "w601sxs/b1ade-embed" ))
# # cosine_sim[0][0]

# # Proper method (pass model in as an argument)
# cosine_sim = cosine_similarity(embed_mesh_headings_preloaded_model(text1, tokenizer= tokenizer, model= model), 
#                                embed_mesh_headings_preloaded_model(text2, tokenizer= tokenizer, model= model))
# cosine_sim[0][0]

# # Proper method (pass model in as an argument)
# cosine_sim = cosine_similarity(embed_mesh_headings_preloaded_model(text1, client= client), 
#                                embed_mesh_headings_preloaded_model(text2, client= client))
# cosine_sim[0][0]

Original semantic similarity check functions (obsolete).

In [14]:
# # Storing the embeddings all of the details returned from the search
# # NEED TO FURTHER DECOUPLE AND MODULARIZE
# def extract_content_and_embeddings_from_retrieved_article_details(retrieved_article_details):
#     heading_entries = []
#     article_entries = []
#     for key, value in retrieved_article_details.items():
#         # Extract and embed all extracted headings
#         for mesh_heading in value['Mesh Headings']:
#             heading_entry = {
#                 'name': mesh_heading['Descriptor'],
#                 'content': descriptors[mesh_heading['Descriptor']]['Combined Description'],
#                 'embeddings': embed_mesh_headings_preloaded_model(descriptors[mesh_heading['Descriptor']]['Combined Description'], tokenizer= tokenizer, model= model)
#             }
#             heading_entries.append(heading_entry)
#         title = value['Title'] or ''
#         abstract = value ['Abstract'] or ''
#         title_abstract = title + abstract
#         article_entry = {
#             'name': key,
#             'content': title_abstract,
#             'embeddings': embed_mesh_headings_preloaded_model(title_abstract, tokenizer= tokenizer, model= model)
#         }
#         article_entries.append(article_entry)
#     return heading_entries, article_entries

# heading_entries, article_entries = extract_content_and_embeddings_from_retrieved_article_details(retrieved_article_details)

# # Compare vector similarity of query to these items
# def extract_filtered_entries_from_content_and_embeddings(heading_entries, article_entries):
#     filtered_heading_entries = []
#     filtered_article_entries = []
#     query_embedding = embed_mesh_headings_preloaded_model(query, tokenizer= tokenizer, model= model)
#     for heading_entry in heading_entries:
#         similarity = cosine_similarity(query_embedding, heading_entry['embeddings'])[0][0]
#         print('Heading similarity is:', similarity)
#         # if  similarity > heading_threshold:
#         if similarity > heading_threshold:
#             heading_entry['Suitability'] = similarity*100
#             filtered_heading_entries.append(heading_entry)
#     for article_entry in article_entries:
#         similarity = cosine_similarity(query_embedding, article_entry['embeddings'])[0][0]
#         print('Article similarity is:', similarity)
#         # if similarity > title_abstract_threshold:
#         if similarity > title_abstract_threshold:
#             article_entry['Suitability'] = similarity*100
#             filtered_article_entries.append(article_entry)
#     return filtered_heading_entries, filtered_article_entries

# filtered_heading_entries, filtered_article_entries = extract_filtered_entries_from_content_and_embeddings(heading_entries, article_entries)


The new manager class that supersedes previous functions.

In [1]:
# class QueryExpansionManager():
#     def __init__(self, model_name, descriptor_json_location, api_key=None,) -> None:
#         if model_name == 'openai':
#             if api_key is None:
#                 raise ValueError("API key is required for OpenAI models")
#             self.model_name = model_name
#             self.client = OpenAI(api_key = api_key)
#             self.tokenizer = None
#             self.model = None
#         else:
#             self.model_name = model_name
#             self.client = None
#             self.tokenizer = AutoTokenizer.from_pretrained(model_name)
#             self.model = AutoModel.from_pretrained(model_name)
#         with open(descriptor_json_location) as json_file:
#             self.descriptors = json.load(json_file)
#         self.query = ''
#         self.translated_query=''
#         self.retrieved_article_details = {}    

#     # This function handles embedding for all the rest of the task in this class 
#     # It is crucial in standardizing the Manager class regardless of embedding models since it returns a unified format that can be used by all other functions.
#     def embed_mesh_headings_preloaded_model(self, text):
#         '''Takes in a text as argument and outputs the embedding vector.'''
#         #if using an openai model
#         if self.model_name == 'openai':
#             text = text.replace("\n", " ")
#             embeddings = np.array([self.client.embeddings.create(input = [text], model="text-embedding-3-large").data[0].embedding])
#         else: 
#             # Tokenize the text and get embeddings
#             inputs = self.tokenizer([text], return_tensors="pt", padding=True, truncation=True)
#             with torch.no_grad():
#                 embeddings = self.model(**inputs).last_hidden_state.mean(dim=1).numpy()  # Use mean pooling for sentence embedding            
#         return embeddings    
    
#     # This function will extract
#     def extract_filtered_entries_from_retrieved_article_details(self, retrieved_article_details, query, title_abstract_threshold, heading_threshold):
#         '''Takes in several arguments and returns the relevant heading entries and article entries, in that order.'''
#         self.retrieved_article_details = retrieved_article_details
#         self.query = query
#         query_embedding = self.embed_mesh_headings_preloaded_model(query)
#         heading_entries = []
#         article_entries = []
#         for key, value in retrieved_article_details.items():
#             article_headings = []
#             # Extract and embed all extracted headings for separate relevance ranking of the terms itself
#             for mesh_heading in value['Mesh Headings']: 
#                 article_headings.append(mesh_heading['Descriptor'])
#                 mesh_heading_embeddings = self.embed_mesh_headings_preloaded_model(descriptors[mesh_heading['Descriptor']]['Combined Description'])   
#                 similarity = cosine_similarity(query_embedding, mesh_heading_embeddings)[0][0]    # output is in fraction
#                 if similarity > heading_threshold:        
#                     heading_entry = {
#                         'name': mesh_heading['Descriptor'],
#                         'content': descriptors[mesh_heading['Descriptor']]['Combined Description'],
#                         'embeddings': mesh_heading_embeddings,
#                         'suitability': similarity*100
#                     }
#                     heading_entries.append(heading_entry)
#             title = value['Title'] or ''
#             abstract = value ['Abstract'] or ''
#             title_abstract = title + abstract
#             article_title_abstract_embeddings = self.embed_mesh_headings_preloaded_model(title_abstract)
#             article_similarity = cosine_similarity(query_embedding, article_title_abstract_embeddings)[0][0]
#             if article_similarity > title_abstract_threshold:
#                 article_entry = {
#                     'name': key,
#                     'content': title_abstract,
#                     'embeddings': article_title_abstract_embeddings,
#                     'headings': article_headings,
#                     'suitability': similarity*100
#                 }
#                 article_entries.append(article_entry)
#         return heading_entries, article_entries 


c:\Users\DataMicron\anaconda3\envs\nlpenv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### This subsection will extract out relevant articles and mesh headings.

Will also extract out the relevant MeSH keywords (using either the set method that aggregates all articles or are relevant to just one)

In [ ]:
from helper.pubmed_search import QueryExpansionManager
from helper.config import api_key
import pandas as pd

model_name1 = "sentence-transformers/all-mpnet-base-v2"
model_name2 = "w601sxs/b1ade-embed"  
# mpnet_qe_manager = QueryExpansionManager(model_name1, 'helper/descriptors.json')
blade_embed_qe_manager = QueryExpansionManager(model_name2, 'helper/descriptors.json')
# openai_qe_manager = QueryExpansionManager('openai', 'helper/descriptors.json', api_key=api_key)

In [3]:
query = "Is creatinine assessment included in the MELD score?"
# mpnet_qe_manager.get_retrieved_article_details_from_query(query)
blade_embed_qe_manager.get_retrieved_article_details_from_query(query)

In [4]:
title_abstract_threshold = 0.3
heading_threshold = 0.5
heading_entries, article_entries = blade_embed_qe_manager.get_filtered_entries_from_retrieved_article_details(title_abstract_threshold, heading_threshold)
if heading_entries!=[]:
    heading_entries_df = pd.DataFrame(heading_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)
else:
    print('Nothing returned!')

Then, for each article within the relevance threshold, we extract out the concepts relevant to it (optionally rank the descriptors based on relevance but no need) and then link them together using 'AND' and enclose them in one bracket as an 'article concept' in a semantic region/viscinity/cluster. In turn, each article is linked to other articles with an 'OR'.

In [9]:
query = blade_embed_qe_manager.create_semantic_neighbourhood_query(heading_entries, article_entries)
query

'AND "Biomarkers"[MeSH Terms] AND "Severity of Illness Index"[MeSH Terms] ) OR ("Amino Acids, Branched-Chain"[MeSH Terms] AND "Dietary Supplements"[MeSH Terms] AND "Muscle Strength"[MeSH Terms] ) OR ("Acute Kidney Injury"[MeSH Terms] ) OR AND "Nomograms"[MeSH Terms] )'

In [8]:
import helper.pubmed_search as pubs
pubs.get_query_response(query)

{'header': {'type': 'esearch', 'version': '0.3'},
 'esearchresult': {'count': '90797',
  'retmax': '5',
  'retstart': '0',
  'idlist': ['39243071', '39242619', '39242503', '39242108', '39240939'],
  'translationset': [],
  'querytranslation': '("Biomarkers"[MeSH Terms] AND "Severity of Illness Index"[MeSH Terms]) OR ("amino acids, branched chain"[MeSH Terms] AND "Dietary Supplements"[MeSH Terms] AND "Muscle Strength"[MeSH Terms]) OR "Acute Kidney Injury"[MeSH Terms] OR "Nomograms"[MeSH Terms]',
  'warninglist': {'phrasesignored': [],
   'quotedphrasesnotfound': [],
   'outputmessages': ['AND', ')']}}}

In [11]:
requery = blade_embed_qe_manager.create_semantic_neighbourhood_query(heading_entries, article_entries)
blade_embed_qe_manager.get_retrieved_article_details_from_query(requery, requery=True, remove_stop_words=False)
requeried_heading_entries, requeried_article_entries = blade_embed_qe_manager.get_filtered_entries_from_retrieved_article_details(0.2, 0.2)
requeried_article_entries_df = pd.DataFrame(requeried_article_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)
requeried_article_entries_df

KeyError: 'querytranslation'

In [23]:
pd.concat([article_entries_df, requeried_article_entries_df]).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False).head(10)['name'].to_list()

['39242688',
 '38125829',
 '38018513',
 '39119581',
 '39242716',
 '38355242',
 '38643268',
 '39243024',
 '38402855',
 '38298766']

In [ ]:
heading_entries, article_entries

In [19]:
semantic_neighbourhoods = []
for article_entry in article_entries:
    expansion_set = article_entry['headings']
    if len(expansion_set)!=0:
        chunk_fragments_to_expand = []
        for i, val in enumerate(expansion_set):
            # First, vet out mesh terms deemed not relevant to the search
            if val not in heading_entries:
                continue
            if i == 0: # first term of set
                chunk_fragments_to_expand.append(f'("{val}"[MeSH Terms]')
            elif i == (len(expansion_set)-1): # final term of set
                chunk_fragments_to_expand.append(f'AND "{val}"[MeSH Terms])')
            else:
                chunk_fragments_to_expand.append(f'AND "{val}"[MeSH Terms]')
        article_semantic_neighbourhood = ' '.join(chunk_fragments_to_expand)
        if article_semantic_neighbourhood!= '':
            semantic_neighbourhoods.append(article_semantic_neighbourhood)
semantic_neighbourhoods

[]

In [21]:
if semantic_neighbourhoods != []:
    combination_of_semantic_neighbourhoods = " OR ".join(semantic_neighbourhoods)
else: 
    combination_of_semantic_neighbourhoods = ''
combination_of_semantic_neighbourhoods

''

In [181]:
expanded_query = translated_query + 'AND (' + combination_of_semantic_neighbourhoods + ')'
expanded_query

'("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields])AND (("Asthma"[MeSH Terms] AND "COVID-19"[MeSH Terms] AND "Exanthema"[MeSH Terms]) OR ("Cough"[MeSH Terms] AND "Exanthema"[MeSH Terms] AND "Sneezing"[MeSH Terms]))'

In [188]:
pubs.get_query_response(combination_of_semantic_neighbourhoods, preprocess=False, articles_to_retrieve=20)

{'header': {'type': 'esearch', 'version': '0.3'},
 'esearchresult': {'count': '4',
  'retmax': '4',
  'retstart': '0',
  'idlist': ['35624503', '33627147', '29795198', '13020435'],
  'translationset': [],
  'querytranslation': '("Asthma"[MeSH Terms] AND "COVID-19"[MeSH Terms] AND "Exanthema"[MeSH Terms]) OR ("Cough"[MeSH Terms] AND "Exanthema"[MeSH Terms] AND "Sneezing"[MeSH Terms])'}}

In [187]:
new_retrieved_article_details, new_translated_query = pubs.extract_article_details_from_query(combination_of_semantic_neighbourhoods, remove_stop_words=False, articles_to_retrieve=20)
new_retrieved_article_details

{'35624503': {'Title': 'Allergic shiners in a patient with cough-variant asthma: a case report.',
  'Abstract': 'Chronic cough, with a duration of coughing of more than 8 weeks in adults, affects 5-10% of the general population. One of the most common causes of chronic cough is cough-variant asthma, which accounts for approximately one-third of cases. This phenotype of asthma is characterized by extreme sensitivity of the neuronal pathways mediating cough to environmental irritants, which results in an urge to cough. This case is an example of cough-variant asthma presenting with allergic shiners due to her severe cough.',
  'Mesh Headings': [{'Descriptor': 'Asthma',
    'Descriptor UI': 'D001249',
    'Is Major Topic': 'Y'},
   {'Descriptor': 'COVID-19',
    'Descriptor UI': 'D000086382',
    'Is Major Topic': 'Y'},
   {'Descriptor': 'Cyprinidae',
    'Descriptor UI': 'D003530',
    'Is Major Topic': 'Y'},
   {'Descriptor': 'Exanthema',
    'Descriptor UI': 'D005076',
    'Is Major To

In [189]:
expanded_heading_entries, expanded_article_entries = blade_embed_qe_manager.extract_filtered_entries_from_retrieved_article_details(new_retrieved_article_details, combination_of_semantic_neighbourhoods, title_abstract_threshold, heading_threshold)
pd.DataFrame(expanded_article_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,headings,suitability
0,35624503,Allergic shiners in a patient with cough-varia...,"[[-0.2013416, 0.4105001, -0.39661187, 0.395849...","[Asthma, COVID-19, Cyprinidae, Exanthema]",63.176656
1,33627147,Distinguishing active pediatric COVID-19 pneum...,"[[-0.0075050797, 0.047646612, -0.38775554, 1.0...",[],63.176656
3,13020435,[Development of normal reflexes and reactions ...,"[[-0.6718215, -0.034178186, 0.235743, -0.10054...","[Cough, Exanthema, Nervous System Physiologica...",59.345675
2,29795198,The association of ambient PMChildren are a su...,"[[-0.38462546, 0.7128517, 0.17911923, 0.227644...",[Absenteeism],50.948042


Next, we extract relevant MeSH keywords and add it to a list for requery.

In [124]:
def extract_query_fragments_and_connectors(s):
    result = []
    temp = ''
    inside_parentheses = False    
    for char in s:
        if char == '(':
            if temp:
                result.append(temp.strip())  # Add content before '(' to the list
                temp = ''
            inside_parentheses = True
            temp += char  # Add '(' to the temp
        elif char == ')':
            temp += char  # Add ')' to the temp
            result.append(temp.strip())  # Add content inside parentheses to the list
            temp = ''
            inside_parentheses = False
        else:
            temp += char    
    if temp:  # If there is any remaining content, add it to the list
        result.append(temp.strip())    
    return result

print(translated_query)
extracted_query_fragments = extract_query_fragments_and_connectors(translated_query)
extracted_query_fragments

("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields])


['("purple"[All Fields] OR "purples"[All Fields])',
 'AND',
 '("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields])',
 'AND',
 '("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields])']

In [142]:
import re
# Example strings
strings = [
    '"Diabetes"[MeSH Terms]',
    '"Hypertension"[MeSH Terms]',
    '"Cancer"[MeSH Terms]',
    'Random text',
    '"Heart Disease"[MeSH Terms] is a condition.',
    '("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields])'
]
pattern = r'"(.*?)"\[MeSH Terms\]'

[re.search(pattern, s).group(1) if re.search(pattern, s) else None for s in strings]

extracted_terms = [re.search(pattern, s).group(1) if re.search(pattern, s) else None for s in strings]
print(extracted_terms)

['Diabetes', 'Hypertension', 'Cancer', None, 'Heart Disease', 'exanthema']

In [140]:
'("purple"[All Fields] OR "purples"[All Fields])'

'("purple"[All Fields] OR "purples"[All Fields])'

In [143]:
def extract_terms_from_translated_set(extracted_query_fragments):
    pattern = r'"(.*?)"\[MeSH Terms\]'
    atm_terms = {}
    for fragment in extracted_query_fragments:
        print(fragment)
        print(type(fragment))
        extracted_terms = [re.search(pattern, s).group(1) if re.search(pattern, s) else None for s in fragment]
        print(extracted_terms)
    #     if extracted_terms is not None:
    #         atm_terms[extracted_terms] = fragment
    # return atm_terms

extract_terms_from_translated_set(extracted_query_fragments)

("purple"[All Fields] OR "purples"[All Fields])
<class 'str'>
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
AND
<class 'str'>
[None, None, None]
("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields])
<class 'str'>
[None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None, None]
AND
<class 'str'>
[None, None, None]
("cough"[MeSH Terms] OR "cough"[A

In [126]:
'"cough"[MeSH Terms]' in extracted_query_fragments[4]

False

In [103]:
expansion_set = set()
for article_entry in article_entries:
    # print(filtered_article['name'])
    for individual_heading_entry in retrieved_article_details[article_entry['name']]['Mesh Headings']:
        expansion_set.add(individual_heading_entry['Descriptor'])
    expansion_set
expansion_set

{'Asthma',
 'COVID-19',
 'Cough',
 'Cyprinidae',
 'Exanthema',
 'Nervous System Physiological Phenomena',
 'Reflex',
 'Sneezing'}

In [104]:
descriptors['Exanthema']

{'Synonyms': ['Exanthema', 'Exanthem', 'Rash', 'Skin Rash', 'Rash, Skin'],
 'Annotation': 'a skin eruption usually chem- or viral-induced; EXANTHEMA SUBITUM is also available\n  ',
 'Scope Note': "Diseases in which skin eruptions or rashes are a prominent manifestation. Classically, six such diseases were described with similar rashes; they were numbered in the order in which they were reported. Only the fourth (Duke's disease), fifth (ERYTHEMA INFECTIOSUM), and sixth (EXANTHEMA SUBITUM) numeric designations survive as occasional synonyms in current terminology.\n    ",
 'Combined Description': "Diseases in which skin eruptions or rashes are a prominent manifestation. Classically, six such diseases were described with similar rashes; they were numbered in the order in which they were reported. Only the fourth (Duke's disease), fifth (ERYTHEMA INFECTIOSUM), and sixth (EXANTHEMA SUBITUM) numeric designations survive as occasional synonyms in current terminology.\n    a skin eruption usua

In [112]:
# expanded_query_fragments = extracted_query_fragments
chunk_fragments_to_expand = []
for i, val in enumerate(expansion_set):
    if i == 0: # first term of set
        chunk_fragments_to_expand.append(f'("{val}"[MeSH Terms]')
    elif i == (len(expansion_set)-1): # final term of set
        chunk_fragments_to_expand.append(f'OR "{val}"[MeSH Terms])')
    else:
        chunk_fragments_to_expand.append(f'OR "{val}"[MeSH Terms]')
chunk_to_expand = ' '.join(chunk_fragments_to_expand)
chunk_to_expand

'("Cough"[MeSH Terms] OR "COVID-19"[MeSH Terms] OR "Cyprinidae"[MeSH Terms] OR "Reflex"[MeSH Terms] OR "Asthma"[MeSH Terms] OR "Exanthema"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms] OR "Sneezing"[MeSH Terms])'

In [114]:
retrieved_expanded_article_details, translated_expanded_query = pubs.extract_article_details_from_query(translated_query + ' AND ' + chunk_to_expand, remove_stop_words=False, articles_to_retrieve=20)
translated_expanded_query

'("purple"[All Fields] OR "purples"[All Fields]) AND ("Exanthema"[MeSH Terms] OR "Exanthema"[All Fields] OR "rashes"[All Fields]) AND ("Cough"[MeSH Terms] OR "Cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields]) AND ("Cough"[MeSH Terms] OR "COVID-19"[MeSH Terms] OR "Cyprinidae"[MeSH Terms] OR "Reflex"[MeSH Terms] OR "Asthma"[MeSH Terms] OR "Exanthema"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms] OR "Sneezing"[MeSH Terms])'

In [123]:
translated_query

'("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields])'

In [120]:
test = '("purple"[All Fields] OR "purples"[All Fields]) AND ("Exanthema"[MeSH Terms] OR "Exanthema"[All Fields] OR "rashes"[All Fields]) AND ("Cough"[MeSH Terms] OR "Cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields]) AND ("COVID-19"[MeSH Terms] OR "Cyprinidae"[MeSH Terms] OR "Reflex"[MeSH Terms] OR "Asthma"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms] OR "Sneezing"[MeSH Terms])'
retrieved_expanded_article_details, translated_expanded_query = pubs.extract_article_details_from_query(chunk_to_expand, remove_stop_words=False, articles_to_retrieve=20)
translated_expanded_query

'"Cough"[MeSH Terms] OR "COVID-19"[MeSH Terms] OR "Cyprinidae"[MeSH Terms] OR "Reflex"[MeSH Terms] OR "Asthma"[MeSH Terms] OR "Exanthema"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms] OR "Sneezing"[MeSH Terms]'

In [121]:
expanded_heading_entries, expanded_article_entries = qe_manager.extract_filtered_entries_from_retrieved_article_details(retrieved_expanded_article_details, query, title_abstract_threshold, heading_threshold)
pd.DataFrame(expanded_heading_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,suitability
2,COVID-19,A viral disorder generally characterized by hi...,"[[-0.010197605, -0.21895505, 0.11601348, -0.09...",43.666178
32,"Immunization, Secondary",Any immunization following a primary immunizat...,"[[0.033557262, -0.31072006, 0.18211477, -0.170...",33.800560
35,Vaccination,Administration of vaccines to stimulate the ho...,"[[0.024296435, -0.23892102, 0.09250141, -0.150...",30.236161
9,"Disease Models, Animal",Naturally-occurring or experimentally-induced ...,"[[0.021185394, -0.026043989, 0.1372653, -0.159...",29.191500
14,COVID-19 Drug Treatment,The use of DRUGS to treat COVID19 or its sympt...,"[[-0.05611608, -0.05003875, 0.037707295, -0.10...",27.319974
1,SARS-CoV-2,A species of BETACORONAVIRUS causing atypical ...,"[[0.053536292, -0.10688641, 0.060441017, -0.12...",24.721570
30,"Antibodies, Viral",Immunoglobulins produced in response to VIRAL ...,"[[0.023101866, -0.1811388, 0.2032994, -0.01521...",24.048373
6,COVID-19 Vaccines,Vaccines or candidate vaccines containing SARS...,"[[0.04259034, -0.112093695, 0.054803435, -0.06...",23.301151
28,Asthma,A form of bronchial disorder with three distin...,"[[-0.100331, -0.28149906, -0.009670261, -0.090...",21.999471
24,"Vaccines, Subunit",Vaccines consisting of one or more antigens th...,"[[0.028799502, -0.0485275, 0.094096035, -0.109...",21.727273


In [122]:
pd.DataFrame(expanded_article_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,suitability
0,39242658,SARS-CoV-2 viral load is linked to remdesivir ...,"[[-0.010255081, -0.09798241, -0.028058521, -0....",17.261556


In [59]:
extracted_query_fragments.append("OR")
extracted_query_fragments.append('"cough"[MeSH Terms]')
test_statement = " ".join(extracted_query_fragments)
test_statement

'("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields]) OR "Cough"[MeSH Terms] OR "cough"[MeSH Terms]'

In [ ]:
"Cough"[MeSH Terms]
"COVID-19"[MeSH Terms]
"Cyprinidae"[MeSH Terms]
"Reflex"[MeSH Terms]
"Nervous System Physiological Phenomena"[MeSH Terms]
"Exanthema"[MeSH Terms]
"Asthma"[MeSH Terms]
"Sneezing"[MeSH Terms]

In [67]:
test_statement = '("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields]) AND ("covid-19"[MeSH Terms] OR "cyprinidae"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms])'
test_article_details, translated_query = pubs.extract_article_details_from_query(test_statement, remove_stop_words=False, articles_to_retrieve=10)
test_heading_entries, test_article_entries = qe_manager.extract_filtered_entries_from_retrieved_article_details(test_article_details, query, title_abstract_threshold, heading_threshold)
pd.DataFrame(test_heading_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,suitability
2,Exanthema,Diseases in which skin eruptions or rashes are...,"[[0.09581368, -0.26140502, 0.117319815, 0.0547...",43.781447
1,COVID-19,A viral disorder generally characterized by hi...,"[[-0.010197605, -0.21895505, 0.11601348, -0.09...",43.666178
0,Asthma,A form of bronchial disorder with three distin...,"[[-0.100331, -0.28149906, -0.009670261, -0.090...",21.999471
3,Cough,"A sudden, audible expulsion of air from the lu...","[[-0.11444584, -0.4871452, 0.07324287, -0.0474...",19.485989
5,Sneezing,"The sudden, forceful, involuntary expulsion of...","[[-0.14127521, -0.43756396, 0.11732302, -0.123...",19.093876


In [68]:
translated_query

'("purple"[All Fields] OR "purples"[All Fields]) AND ("exanthema"[MeSH Terms] OR "exanthema"[All Fields] OR "rashes"[All Fields]) AND ("cough"[MeSH Terms] OR "cough"[All Fields] OR "coughing"[All Fields] OR "coughs"[All Fields] OR "coughed"[All Fields]) AND ("covid-19"[MeSH Terms] OR "cyprinidae"[MeSH Terms] OR "Nervous System Physiological Phenomena"[MeSH Terms])'

In [69]:
pd.DataFrame(test_article_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,suitability
0,35624503,Allergic shiners in a patient with cough-varia...,"[[0.010772239, -0.1761123, 0.07280753, 0.05663...",43.781447
1,13020435,[Development of normal reflexes and reactions ...,"[[-0.049132317, -0.15629517, 0.13611218, -0.01...",19.093876


In [58]:
pd.DataFrame(test_article_entries).drop_duplicates(['name']).sort_values(by=['suitability'], ascending=False)

,name,content,embeddings,suitability
3,39217183,Mycobacterium tuberculosis cough aerosol cultu...,"[[0.046328314, 0.047822025, -0.049906746, 0.00...",37.748751
1,39222970,Chronic cough in preschool aged children.,"[[0.016885558, 0.18008955, 0.058235824, -0.001...",19.485989
2,39219607,Cough-induced severe bradycardia and syncope i...,"[[-0.0027521388, -0.032782163, 0.059245188, -0...",5.422820
0,39235438,Content validity of the Leicester Cough Questi...,"[[0.03931393, 0.15439567, 0.018702619, -0.0283...",-8.319442


In [73]:
pd.DataFrame(filtered_article_entries).drop_duplicates(['name']).sort_values(by=['Suitability'], ascending=False)

,name,content,embeddings,Suitability
1,28759392,"A 3-Week-Old With an Isolated ""Blueberry Muffi...","[[0.030234266, -0.13129903, 0.12427848, -0.070...",47.270375
2,13020435,[Development of normal reflexes and reactions ...,"[[-0.049132317, -0.15629517, 0.13611218, -0.01...",46.364245
0,35624503,Allergic shiners in a patient with cough-varia...,"[[0.010772239, -0.1761123, 0.07280753, 0.05663...",42.842036


Requery stage....not sure how many articles are relevant.

In [37]:
test_article_details = pubs.extract_article_details_from_query(test_statement, remove_stop_words=False, articles_to_retrieve=10)
heading_entries_test, article_entries_test = extract_content_and_embeddings_from_retrieved_article_details(test_article_details)
filtered_heading_entries_test, filtered_article_entries_test = extract_filtered_entries_from_content_and_embeddings()

({'39238401': {'Title': '[Expert recommendations on the clinical application of mechanical insufflation-exsufflation in mechanically ventilated patients (2024)].',
   'Abstract': 'Mechanical ventilated patients are a high-risk group with impaired cough ability and require corresponding medical techniques for cough assistance to clear airway secretions. Mechanical insufflation-exsufflation (MI-E) technology is widely used in patients with cough weakness caused by neuromuscular diseases. However, there is currently a lack of standardized application procedures for mechanically ventilated patients who retain artificial airways, which can affect treatment outcomes. Chinese Society of Critical Care Medicine organized experts including critical care physicians, nurses, respiratory therapists that focused on the clinical application of mechanical insufflation-exsufflation in mechanically ventilated patients. Through systematic collection, extraction, and summary of evidence-based clinical pra

## Manually Extracting Query

In [111]:
with open('helper/descriptors.json') as json_file:
    descriptors = json.load(json_file)
with open('helper/concepts.json') as json_file:
    concepts = json.load(json_file)
with open('helper/terms.json') as json_file:
    terms = json.load(json_file)

Strategy:
1. Each word is broken down into semantic chunks
2. For each semantic chunk, extract descriptor, term and preferred term (put them in an or chunk)
3. Link all chunks together via AND
4. If no articles returned, then have an option to remove words one by one
5. (GET TO THIS PART QUICK) For each returned article, score them and see if the words associated with the article are synonyms with some descriptor terms.
6. Suggest those additional terms to the user

Examples:
1. 'term=science[journal]+AND+breast+cancer+AND+2008[pdat]'
2. '"skin"[MeSH Terms] OR "rashes"[MeSH Terms] OR "purple"[MeSH Terms]'

In [112]:
import requests
import spacy
from nltk.corpus import stopwords
from rapidfuzz import process, fuzz

threshold = 90
# Load SpaCy's language model
nlp = spacy.load("en_core_web_sm")

# Define stop words using NLTK
stop_words = set(stopwords.words('english'))

def remove_stop_words(query):
    doc = nlp(query)
    # Filter out stop words and only keep nouns and adjectives
    filtered_tokens = [token.lemma_ for token in doc if token.pos_ in ["NOUN", "ADJ"] and token.lemma_.lower() not in stop_words]
    # return " ".join(filtered_tokens)
    return filtered_tokens

# Checks for each term in the query
search_tokens = remove_stop_words('I have some purple rashes and cough.')


def get_term_groupings_from_tokens(search_tokens, descriptors, concepts, terms):
    expanded_search_tokens = {}
    mandatory_terms = []
    optional_terms = []
    not_present_terms = []
    # Checks whether query exists in descriptor layer
    for token in search_tokens:
        print('Token is:', token)
        descriptor_key_match = process.extractOne(token.capitalize(), descriptors.keys(), score_cutoff=threshold, scorer=fuzz.ratio)
        if descriptor_key_match:
            print(f"Best match in descriptors: {descriptor_key_match[0]} with score: {descriptor_key_match[1]}")
            print(descriptors[descriptor_key_match[0]])
            expanded_search_tokens[token] = {
                'descriptor' : descriptor_key_match[0]
            }
            # mandatory_terms.append (descriptor_key_match[0])
            continue
        # If not found, then move on to concept layer
        # If found, then extract the preferred term and see whether it is in descriptor layer - put current term on 'or'
        concept_key_match = process.extractOne(token.capitalize(), concepts.keys(), score_cutoff=threshold, scorer=fuzz.ratio)
        if concept_key_match:
            print(f"Best match in concepts: {concept_key_match[0]} with score: {concept_key_match[1]}")
            print(concepts[concept_key_match[0]])
            
            # optional_terms.append(concept_key_match[0])
            continue
        # Else, move to term layer
        # If found, then find preferred term and see whether it is in descriptor layer - put current term on 'or'
        terms_key_match = process.extractOne(token.capitalize(), terms.keys(), score_cutoff=threshold, scorer=fuzz.ratio)
        if terms_key_match:
            print(f"Best match in terms: {terms_key_match[0]} with score: {terms_key_match[1]}")
            print(terms[terms_key_match[0]])
            optional_terms.append(terms_key_match[0])
            continue
        not_present_terms.append(token)

# Add additional query suggestion for terms similar to what user is asking (spelling check query suggestion)
query = ''

first_term_inserted = False
# If mesh term, then search mesh term only
def build_query_from_tokens(mandatory_terms, optional_terms, not_in_mesh_terms):
    query = ''
    for mandatory_term in mandatory_terms:
        if first_term_inserted:
            query+=' AND '
        query+='"'+ mandatory_term + '"' + '[MeSH Terms]'
        first_term_inserted = True
    # If got concept, expand by preferred concept (all in an 'OR')
    # Also expand (maybe an 'OR') by the cocept
    for optional_term in optional_terms:
        if first_term_inserted:
            query+=' OR '
        query+='"'+ optional_term + '"' + '[All Fields]'
    for not_present_term in not_in_mesh_terms:
        if first_term_inserted:
            query+=' OR '
        query+='"'+ not_present_term + '"' + '[All Fields]'
    # '"skin"[MeSH Terms] OR "rashes"[MeSH Terms] OR "purple"[MeSH Terms]'
    # If exists in descriptor, move to next stage
    # Create new query. For words totally not in either layer, put it on 'term search'
    # Else, 
    query

This code that extracts information straight from the internet is no longer relevant.

In [113]:
# import requests

# def get_mesh_details(term):
#     base_url = "https://id.nlm.nih.gov/mesh/"
#     search_url = f"{base_url}/lookup/descriptor?label={term}&match=contains"

#     response = requests.get(search_url)
#     if response.status_code == 200:
#         data = response.json()
#         return data
#     else:
#         print(f"Error: {response.status_code}")
#         return None

# # Example usage
# term = "rash"
# mesh_details = get_mesh_details(term)

# if mesh_details:
#     for item in mesh_details:
#         print(f"Label: {item['label']}")
#         print(f"ID: {item['resource']}")
#         print(f"Tree Numbers: {item.get('treeNumbers', 'N/A')}")
#         print()



In [224]:
# import helper.pubmed_search as pubs
# import random
# import time
# import json
# import math

# with open ('evaluation/BioASQ/BioASQ-training11b/training11b.json') as training_file:
#     full_evaluation_data = json.load(training_file)

# # Actually same as precision at k
# # def simple_precision(retrieved_documents, relevant_documents):
# #     # Convert to sets
# #     set1 = set(retrieved_documents)
# #     set2 = set(relevant_documents)
# #     # Find the overlap (intersection)
# #     overlap = set1 & set2
# #     # If you want the size of the overlap
# #     overlap_size = len(overlap)
# #     try:
# #         precision = overlap_size/len(retrieved_documents)*100
# #     except Exception as e:
# #         print('Error at measure_pak function:', e)        
# #         precision = 0
# #     return precision

# def precision_at_k(retrieved_docs, relevant_docs, k=10):
#     k = min(k, len(retrieved_docs))  # Handle case where retrieved_docs < k
#     relevant_in_top_k = [doc for doc in retrieved_docs[:k] if doc in relevant_docs]
#     return 100*len(relevant_in_top_k) / k if k > 0 else 0.0


# def average_precision_at_k(retrieved_docs, relevant_docs, k=10):
#     k = min(k, len(retrieved_docs))  # Handle case where retrieved_docs < k
#     if k == 0:
#         return 0.0

#     num_relevant = 0
#     precision_sum = 0
    
#     for i in range(1, k + 1):
#         if retrieved_docs[i - 1] in relevant_docs:
#             num_relevant += 1
#             precision_sum += num_relevant / i
    
#     # Use the smaller of k or total number of relevant documents
#     return 100*precision_sum / min(len(relevant_docs), k) if num_relevant > 0 else 0.0

# def ndcg_at_k(retrieved_docs, relevant_docs, k=10):
#     k = min(k, len(retrieved_docs))  # Handle case where retrieved_docs < k
#     if k == 0:
#         return 0.0

#     def dcg(retrieved_docs, relevant_docs, k):
#         dcg_value = 0.0
#         for i in range(k):
#             if retrieved_docs[i] in relevant_docs:
#                 dcg_value += 1 / math.log2(i + 2)  # i + 2 because of 0-based indexing
#         return dcg_value

#     def idcg(relevant_docs, k):
#         idcg_value = 0.0
#         for i in range(min(len(relevant_docs), k)):
#             idcg_value += 1 / math.log2(i + 2)
#         return idcg_value
    
#     dcg_value = dcg(retrieved_docs, relevant_docs, k)
#     idcg_value = idcg(relevant_docs, k)
    
#     return 100*dcg_value / idcg_value if idcg_value > 0 else 0.0


# def extract_subset_of_evaluation_data(evaluation_data, fraction = 0.5, random_seed = 31):   
#     random.seed(random_seed) 
#     subset = random.sample(evaluation_data, int(len(evaluation_data)*fraction))
#     with open ('evaluation/BioASQ/subset.json', 'w') as outfile:
#         json.dump(subset, outfile)
#     print('Extracted', len(subset), 'documents out of', len(evaluation_data))

# # Method to rank relevance of documents
# def rank_initial_relevance(evaluation_set):
#     # evaluation_set = []
#     # if n_samples!=0:
#     #     evaluation_set = random.sample(evaluation['questions'], n_samples)
#     # else:        
#     #     evaluation_set = evaluation['questions']
#     # precisions = []
#     retrieved_ids_list = []
#     ground_truth_list = []
#     # Run it through the system:
#     for entry in evaluation_set:
#         # When running the evaluation, there is a tendency for the internet to break so we need to continuously try to connnect
#         rerun = True
#         runs = 0
#         while rerun:
#             time.sleep(1) # rest for 1 second so as not to overwhelm PubMed's API
#             # query the question through the database and extract the relevant documents
#             try:
#                 query_response = pubs.get_query_response(entry['body'], preprocess=False, articles_to_retrieve=10)
#                 retrieved_ids = query_response['esearchresult']['idlist']
#                 ground_truth = [x.split('/')[-1] for x in entry['documents']]    
#                 retrieved_ids_list.append(retrieved_ids)    
#                 ground_truth_list.append(ground_truth)   
#                 rerun=False # once information is extracted, stop the rerun
#             except Exception as e:
#                 print('Error occured when querying pubmed:', e)
#                 runs +=1
#                 if runs > 5: # after 10 rounds of rerun due to errors, just move on
#                     continue 
#                 rerun=True # if somehow information is not being able to be obtained, keep trying
#     #     if len(retrieved_ids)!=0:
#     #         precision = precision_at_k(retrieved_ids, ground_truth)
#     #         print('Precision is:', precision)
#     #     else:
#     #         precision = 0
#     #         print('No articles returned!')
#     #     precisions.append(precision)
#     # overall_precision = sum(precisions)/len(precisions)
#     return retrieved_ids_list, ground_truth_list

## Extract MeSH Ontology Details into Usable Form (Already Done)

This improved function will return the preferred concept as well as term and descriptor objects to experiment with (**not used**).

In [ ]:
# import xml.etree.ElementTree as ET

# def find_term_in_mesh(xml_file, search_term):
#     tree = ET.parse(xml_file)
#     root = tree.getroot()    
#     term_objects = []
#     descriptor_objects = []
#     list_of_preferred_concept = []
#     for descriptor_record in root.findall('.//DescriptorRecord'):
#         descriptor_name = descriptor_record.find('.//DescriptorName/String').text

#         # Identify the preferred concept within the descriptor
#         preferred_concept = None
#         for concept in descriptor_record.findall('.//Concept'):
#             if concept.attrib.get('PreferredConceptYN', 'N') == 'Y':
#                 preferred_concept = concept.find('.//ConceptName/String').text
#                 description = 'Term '+concept.find('.//String').text+' has preferred concept: '+preferred_concept
#                 list_of_preferred_concept.append(description)
#                 break

#         for concept in descriptor_record.findall('.//Concept'):
#             concept_name = concept.find('.//ConceptName/String').text

#             # Identify the preferred term within the concept
#             preferred_term = None
#             for term in concept.findall('.//Term'):
#                 # if term.attrib.get('PreferredTermYN', 'N') == 'Y':  # for all terms within a concept, find the preferred term in the concept tree
#                 if term.attrib.get('ConceptPreferredTermYN', 'N') == 'Y':  # for all terms within a concept, find the preferred term in the concept tree
#                     preferred_term = term.find('.//String').text
#                     break
#             for term in concept.findall('.//Term'):
#                 term_name = term.find('.//String').text
#                 # if fuzz.ratio(search_term.lower(), term_name.lower()) > 80:
#                 if search_term.lower() == term_name.lower():
#                 # if search_term.lower() in term_name.lower():
#                     print(f"Found Term: {term_name}")
#                     print(f"Descriptor: {descriptor_name}")
#                     print(f"Preferred Concept: {preferred_concept}")
#                     print(f"Concept: {concept_name}")
#                     print(f"Preferred Term: {preferred_term}")
#                     print("-" * 40)
#                     term_objects.append(term)
#                     descriptor_objects.append(descriptor_record)

#     print(f"Search complete for term '{search_term}'.")
#     return term_objects, descriptor_objects, list_of_preferred_concept

# # Example usage
# xml_file_path = 'record/desc2024.xml'  # Replace with your file path
# search_term = 'rash'
# a, b, c = find_term_in_mesh(xml_file_path, search_term)

Code fragment for extracting annotation as well as scope (**not used**).

In [12]:
# import xml.etree.ElementTree as ET
# #  Example function to extract descriptors and descriptions
# def extract_descriptor_details(xml_file):
#     # Parse the XML file
#     tree = ET.parse(xml_file)
#     root = tree.getroot()
#     descriptors = []
    
#     for descriptor in root.findall(".//DescriptorRecord"):
#         descriptor_name = descriptor.findtext("DescriptorName/String")
#         description = descriptor.findtext("ScopeNote")  # ScopeNote usually contains the description
#         scope_note = descriptor.findtext('.//ScopeNote')
#         annotation = descriptor.findtext('.//Annotation')
        
#         descriptors.append({
#             'DescriptorName': descriptor_name,
#             'Description': description,
#             'scope_note': scope_note,
#             'annotation': annotation
#         })
    
#     return descriptors

# # Extract descriptors and descriptions
# descriptors = extract_descriptor_details('record/desc2024.xml')

When building the index, all we need to know is what the preferred concept/term of a phrase is, as well as the related description. If got time, then only we build other things like narrowing and generalization (**final used function**, commented after we manage to retrieve proper index)

In [71]:
# import xml.etree.ElementTree as ET
# from collections import defaultdict
# import json

# def create_mesh_preferred_index(xml_file):
#     # Dictionary to hold the index
#     descriptor_index = defaultdict(dict) # can include synonyms etc. in this
#     concept_index = defaultdict(dict)
#     term_index = defaultdict(dict)
    
#     # Parse the XML file
#     tree = ET.parse(xml_file)
#     root = tree.getroot()

#     # Traverse the XML and build the index
#     for descriptor in root.findall('.//DescriptorRecord'):
#         descriptor_name = descriptor.find('.//DescriptorName/String').text
#         scope_note = descriptor.findtext('.//ScopeNote') or ''
#         annotation = descriptor.findtext('.//Annotation') or ''
#         combined_description = scope_note + annotation 
#         synonyms = [term.find("String").text for term in descriptor.findall(".//TermList/Term")]
#         # DESCRIPTOR LEVEL: The values for the index should be the synonyms and descriptions
#         descriptor_index[descriptor_name] = {
#             'Synonyms' : synonyms,
#             'Annotation': annotation,
#             'Scope Note': scope_note,
#             'Combined Description': combined_description
#         }
#         # CONCEPT LEVEL: First need to find the preferred concept within this descriptor. Need to iterate through all
#         preferred_concept = ''
#         for concept in descriptor.findall('.//Concept'):
#             if concept.attrib.get('PreferredConceptYN', 'N') == 'Y':
#                 preferred_concept = concept.find('.//ConceptName/String').text
#         # Then go to the next step by documenting all concepts and move to next loop
#         for concept in descriptor.findall('.//Concept'):
#             concept_name = concept.find('.//ConceptName/String').text
#             concept_index[concept_name] = {
#                 'preferred concept': preferred_concept,
#                 'descriptor': descriptor_name
#             }
#             # TERM LEVEL: Just like for concepts, need to first find the preferred term
#             preferred_term = ''
#             for term in concept.findall('.//Term'):
#                 # if term.attrib.get('PreferredConceptYN', 'N') == 'Y':
#                 if term.attrib.get('ConceptPreferredTermYN', 'N') == 'Y':
#                     preferred_term = term.find('.//String').text
#             # Then, resume iteration
#             for term in concept.findall('.//Term'):
#                 term_name = term.find('.//String').text
#                 term_index[term_name] = {
#                     'preferred term': preferred_term,
#                     'concept': concept_name,
#                     'descriptor': descriptor_name
#                 }
#     return descriptor_index, concept_index, term_index

# # Example usage
# xml_file_path = 'record/desc2024.xml'
# descriptors, concepts, terms = create_mesh_preferred_index(xml_file_path)

# with open("helper/descriptors.json", "w") as outfile: 
#     json.dump(descriptors, outfile)
# with open("helper/concepts.json", "w") as outfile: 
#     json.dump(concepts, outfile)
# with open("helper/terms.json", "w") as outfile: 
#     json.dump(terms, outfile)


# BRANCH 2: FIND PAPER LINKAGES

In [ ]:
import requests
from xml.etree import ElementTree

# Define the paper's PubMed ID
pubmed_id = '37936304'

# Fetch paper details from PubMed
url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pubmed_id}&retmode=xml'
response = requests.get(url)
tree = ElementTree.fromstring(response.content)

# Extract authors
authors = []
for author in tree.findall(".//Author"):
    last_name = author.find("LastName").text if author.find("LastName") is not None else ""
    fore_name = author.find("ForeName").text if author.find("ForeName") is not None else ""
    authors.append(f"{fore_name} {last_name}")

print("Authors:", authors)


Authors: ['Jamie L Karch', 'Chiamaka L Okorie', 'Mayra B C Maymone', 'Melissa Laughter', 'Neelam A Vashi']


In [ ]:
import requests
from xml.etree import ElementTree

# Step 1: Fetch the author's name from the original paper
pubmed_id = '37936304'
url = f'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={pubmed_id}&retmode=xml'
response = requests.get(url)
tree = ElementTree.fromstring(response.content)

# Extract the first author (you can extract more as needed)
author = tree.find(".//Author")
last_name = author.find("LastName").text
initials = author.find("Initials").text
author_query = f"{last_name} {initials}[Author]"

# Step 2: Search for other papers by this author
search_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=pubmed&term={author_query}&retmode=json"
search_response = requests.get(search_url).json()
pubmed_ids = search_response.get("esearchresult", {}).get("idlist", [])

# Step 3: Fetch details of these papers
if pubmed_ids:
    id_string = ",".join(pubmed_ids)
    fetch_url = f"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=pubmed&id={id_string}&retmode=xml"
    details_response = requests.get(fetch_url)
    details_tree = ElementTree.fromstring(details_response.content)
    
    # Extract titles of these papers as an example
    titles = [article.findtext("ArticleTitle") for article in details_tree.findall(".//Article")]
    print("Titles of other papers by the author:", titles)
else:
    print("No other papers found for this author.")


Titles of other papers by the author: ['Neonatal nasal necrosis: Case series and brief review.', 'Vascular cutaneous manifestations of COVID-19 and RNA viral pathogens: a systematic review.', 'Subcutaneous Nodule With Poliosis: An Unusual Presentation of Melanoma Ex Blue Nevus.']
